In [1]:
import json

In [8]:
# Read training data

fp_hotpot_mdr = "../multihop_dense_retrieval/data/hotpot/hotpot_train_with_neg_v0.json"
fp_hotpot_original = "../multihop_dense_retrieval/data/hotpot/hotpot_train_v1.1.json"

with open(fp_hotpot_mdr, 'r') as handle:
    hotpot_mdr = json.load(handle)
    
with open(fp_hotpot_original, 'r') as handle:
    hotpot_orig = json.load(handle)

In [12]:
hotpot_mdr = []
with open(fp_hotpot_mdr, 'r') as handle:
    lines = handle.readlines()
    for l in lines:
        hotpot_mdr.append(json.loads(l))

In [10]:
# Training data length
len(hotpot_orig)

90447

In [11]:
# Example training object
hotpot_orig[0]

{'supporting_facts': [["Arthur's Magazine", 0], ['First for Women', 0]],
 'level': 'medium',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'context': [['Radio City (Indian radio station)',
   ["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.',
    ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.',
    ' The Radio station currently plays a mix of Hindi and Regional music.',
    ' Abraham Thomas is the CEO of the company.']]

In [16]:
# Count number of questions for which the context is not available for the SUP sentences

count_one, count_both, not_two = 0,0,0
for h in hotpot_orig:
    sup_set = set()
    for sup in h['supporting_facts']:
        sup_set.add(sup[0])
    context_set = set()
    for context in h['context']:
        context_set.add(context[0])
    if len(sup_set.intersection(context_set)) == 0:
        count_both += 1
    if len(sup_set) <2:
        not_two += 1
print(count_both)
print(not_two)

0
0


In [18]:
# Check if the MDR-processed hotpot data is the same length
len(hotpot_mdr)

90447

In [22]:
# Example object in MDR hotpot
hotpot_mdr[1]

{'question': 'The Oberoi family is part of a hotel company that has a head office in what city?',
 'answers': ['Delhi'],
 'type': 'bridge',
 'pos_paras': [{'title': 'Oberoi family',
   'text': 'The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group.'},
  {'title': 'The Oberoi Group',
   'text': 'The Oberoi Group is a hotel company with its head office in Delhi. Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands.'}],
 'neg_paras': [{'title': 'Future Fibre Technologies',
   'text': 'Future Fibre Technologies (FFT) is a fiber optic sensing technologies company based in Melbourne, Australia, with its US head office in Mountain View, California, Middle East head office in Dubai, Indian head office in New Delhi and European head office in London. Founded in 1994, Future Fibre Technologies product line 

In [29]:
def extract_sentences(sup_idx_list, context_list, doc_title=None):
    return_obj = []
    sup = {}
    for s in sup_idx_list:
        if s[0] not in sup:
            sup[s[0]] = []
        sup[s[0]].append(s[1])
    for c in context_list:
        if c[0] not in sup:
            continue
        if doc_title is not None and c[0] != doc_title:
            continue
        for i in sup[c[0]]:
            try:
                return_obj.append(c[1][i])
            except:
                print("IndexError: ignoring")
                print(c[0])
                print(c[1])
                print(sup[c[0]])
    return return_obj

In [30]:
output = []

for i in range(len(hotpot_orig)):
    orig_obj = hotpot_orig[i]
    mdr_obj = hotpot_mdr[i]
    q_obj = {
        "id": orig_obj["_id"],
        "question": orig_obj["question"],
        "level": orig_obj["level"],
        "type": orig_obj["type"],
        "answers": [],
        "final_answer": orig_obj["answer"],
    }
    if orig_obj["type"] == "comparison":
        # Fetch all SUP sentences into "answers"
        q_obj["answers"] = extract_sentences(orig_obj["supporting_facts"], orig_obj["context"])
    else:
        # Fetch only the first-hop SUP sentences into "answers"
        for sup in orig_obj["supporting_facts"]:
            if sup[0] != mdr_obj["bridge"]:
                q_obj["answers"] = extract_sentences(orig_obj["supporting_facts"], orig_obj["context"], sup[0])
                break
    output.append(q_obj)

IndexError: ignoring
Minoru Suzuki
['Minoru Suzuki (鈴木 実 , Suzuki Minoru , ring name: 鈴木 みのる) (born June 17, 1968) is a Japanese professional wrestler and mixed martial artist who is currently working for New Japan Pro Wrestling (NJPW) as a freelancer.', ' He is the current NEVER Openweight Champion in his first reign.']
[0, 1, 2]
IndexError: ignoring
Khady Sylla
['Khady Sylla (Dakar, March 27, 1963 – Dakar, October 8, 2013) was a Senegalese writer of two novels, short work, and film.']
[0, 2]
IndexError: ignoring
Harry Styles
['Harry Edward Styles (born 1 February 1994) is an English singer, songwriter, and actor.', ' He made his debut as a singer with his band White Eskimo, who performed locally in Holmes Chapel, Cheshire.', ' Styles rose to stardom as a member of the boy band One Direction, formed in 2010 through the British music competition series "The X Factor".', ' One Direction has released five albums, performed four worldwide tours, and won several awards.']
[0, 3, 4]
IndexEr

In [33]:
output[19]

{'id': '5ab698885542995eadef002a',
 'question': 'American politician Joe Heck ran unsuccessfully against Democrat Catherine Cortez Masto, a woman who previously served as the 32nd Attorney General of where?',
 'level': 'easy',
 'type': 'bridge',
 'answers': ["Joseph John “Joe” Heck (born October 30, 1961) is an American politician, physician, and U.S. Army Brigadier General who had served as the U.S. Representative for Nevada's 3rd congressional district from 2011 to 2017.",
  ' He ran unsuccessfully against Democrat Catherine Cortez Masto in the general election for the open Nevada United States Senate seat in 2016.'],
 'final_answer': 'Nevada'}

In [35]:
# Store results to disk

output_fp = "../multihop_dense_retrieval/data/hotpot/hotpot_train_firsthop.json"
with open(output_fp, 'w') as fp:
    json.dump({"data": output}, fp, indent=4)